# Link Prediction Baselines
---
ROC AUC and Average Precision computed on YouTube dataset using these link prediction baselines:
1. Adamic-Adar
2. Jaccard Coefficient
3. Preferential Attachment
4. Resource Allocation

## 1. Read in Graph Data

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle

In [2]:
EGO_USER = 0 # which ego network to look at

def read_data_file_as_csr_matrix(filename):
    data = pd.read_csv(filename, sep=' ', header=None, dtype=np.uint32, error_bad_lines = False)
    rows = data[0]
    cols = data[1]
    ones = np.ones(len(rows), np.uint32)
    matrix = sp.csr_matrix((ones, (rows, cols)))
    print(matrix)
    return matrix

# Load pickled (adj, feat) tuple
network_dir = './youtube-data/080327_3.txt_train.txt'
# adj2 = read_data_file_as_csr_matrix(network_dir)

g = nx.Graph()

with open(network_dir, 'r') as f:
    for node_connect in f.readlines():
        node_connect = node_connect.replace('\n', '')
        node_connect = node_connect.split(" ")
        node_connect[0] = int(node_connect[0])
        node_connect[1] = int(node_connect[1])
        g.add_edge(node_connect[0], node_connect[1])
    
        

In [3]:
# draw network
# nx.draw_networkx(g, with_labels=False, node_size=1, node_color='r')
# plt.show()

## 2. Preprocessing/Train-Test Split

In [4]:
from gae.preprocessing import mask_test_edges
np.random.seed(0) # make sure train-test split is consistent between notebooks
adj_sparse = nx.to_scipy_sparse_matrix(g)

# Perform train-test split
adj_train, train_edges, train_edges_false, val_edges, val_edges_false, \
    test_edges, test_edges_false = mask_test_edges(adj_sparse, test_frac=.3, val_frac=.1)
g_train = nx.from_scipy_sparse_matrix(adj_train) # new graph object with only non-hidden edges

Num. (test, val) edges requested: ( 1258 ,  419 )
Num. (test, val) edges returned: ( 537 ,  0 )


In [5]:
# Inspect train/test split
print("Total nodes:", adj_sparse.shape[0])
print("Total edges:", int(adj_sparse.nnz/2)) # adj is symmetric, so nnz (num non-zero) = 2*num_edges)
print("Training edges (positive):", len(train_edges))
print("Training edges (negative):", len(train_edges_false))
print("Validation edges (positive):", len(val_edges))
print("Validation edges (negative):", len(val_edges_false))
print("Test edges (positive):", len(test_edges))
print("Test edges (negative):", len(test_edges_false))

Total nodes: 3949
Total edges: 4194
Training edges (positive): 3657
Training edges (negative): 3657
Validation edges (positive): 0
Validation edges (negative): 419
Test edges (positive): 537
Test edges (negative): 1258


In [6]:
def get_roc_score(edges_pos, edges_neg, score_matrix):
    # Store positive edge predictions, actual values
    preds_pos = []
    pos = []
    for edge in edges_pos:
        preds_pos.append(score_matrix[edge[0], edge[1]]) # predicted score
        pos.append(adj_sparse[edge[0], edge[1]]) # actual value (1 for positive)
        
    # Store negative edge predictions, actual values
    preds_neg = []
    neg = []
    for edge in edges_neg:
        preds_neg.append(score_matrix[edge[0], edge[1]]) # predicted score
        neg.append(adj_sparse[edge[0], edge[1]]) # actual value (0 for negative)
        
    # Calculate scores
    preds_all = np.hstack([preds_pos, preds_neg])
    labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)
    return roc_score, ap_score

## 3. Adamic-Adar

In [7]:
# Compute Adamic-Adar indexes from g_train
import time
start_time = time.time()
aa_matrix = np.zeros(adj_sparse.shape)
for u, v, p in nx.adamic_adar_index(g_train): # (u, v) = node indices, p = Adamic-Adar index
    aa_matrix[u][v] = p
    aa_matrix[v][u] = p # make sure it's symmetric
    
# Normalize array
aa_matrix = aa_matrix / aa_matrix.max()

In [8]:
# Calculate ROC AUC and Average Precision
aa_roc, aa_ap = get_roc_score(test_edges, test_edges_false, aa_matrix)
calc_time = round(time.time() - start_time, 2)
print('Adamic-Adar Test ROC score: ', str(aa_roc))
print('Adamic-Adar Test AP score: ', str(aa_ap))
print('Time taken: {} sec'.format(calc_time))

Adamic-Adar Test ROC score:  0.6041157819008625
Adamic-Adar Test AP score:  0.4430262260870411
Time taken: 94.47 sec


## 4. Jaccard Coefficient

In [9]:
# # Compute Jaccard Coefficients from g_train
jc_matrix = np.zeros(adj_sparse.shape)
start_time = time.time()
for u, v, p in nx.jaccard_coefficient(g_train): # (u, v) = node indices, p = Jaccard coefficient
    jc_matrix[u][v] = p
    jc_matrix[v][u] = p # make sure it's symmetric
    
# Normalize array
jc_matrix = jc_matrix / jc_matrix.max()

In [10]:
# Calculate ROC AUC and Average Precision
jc_roc, jc_ap = get_roc_score(test_edges, test_edges_false, jc_matrix)
calc_time = round(time.time() - start_time, 2)
print('Jaccard Coefficient Test ROC score: ', str(jc_roc))
print('Jaccard Coefficient Test AP score: ', str(jc_ap))
print('Time taken: {} sec'.format(calc_time))

Jaccard Coefficient Test ROC score:  0.6039270456786067
Jaccard Coefficient Test AP score:  0.4347536417524206
Time taken: 124.52 sec


## 5. Preferential Attachment

In [11]:
# Calculate, store Adamic-Index scores in array
pa_matrix = np.zeros(adj_sparse.shape)
start_time = time.time()
for u, v, p in nx.preferential_attachment(g_train): # (u, v) = node indices, p = Jaccard coefficient
    pa_matrix[u][v] = p
    pa_matrix[v][u] = p # make sure it's symmetric
    
# Normalize array
pa_matrix = pa_matrix / pa_matrix.max()

In [12]:
# Calculate ROC AUC and Average Precision
pa_roc, pa_ap = get_roc_score(test_edges, test_edges_false, pa_matrix)
calc_time = round(time.time() - start_time, 2)
print('Preferential Attachment Test ROC score: ', str(pa_roc))
print('Preferential Attachment Test AP score: ', str(pa_ap))
print('Time taken: {} sec'.format(calc_time))

Preferential Attachment Test ROC score:  0.8608606963848503
Preferential Attachment Test AP score:  0.6532588653931537
Time taken: 52.89 sec


# 6. Common Neighbors

For common neighbors, we would explore two different variations of common neighbors. However, it is omitted because we do not have 'community' information as that would require separate evaluation techniques.

1. Ratio of within- and inter-cluster common neighbors of all node pairs in ebunch.
2. Count the number of common neighbors of all node pairs in ebunch using community information.

# 7. Resource Allocation

In [13]:
# Calculate, store Adamic-Index scores in array
ra_matrix = np.zeros(adj_sparse.shape)
start_time = time.time()
for u, v, p in nx.resource_allocation_index(g_train): # (u, v) = node indices, p = Jaccard coefficient
    ra_matrix[u][v] = p
    ra_matrix[v][u] = p # make sure it's symmetric
    
# Normalize array
ra_matrix /= ra_matrix.max()

In [14]:
# Calculate ROC AUC and Average Precision
ra_roc, ra_ap = get_roc_score(test_edges, test_edges_false, ra_matrix)
calc_time = round(time.time() - start_time, 2)
print('Resource Allocation Test ROC score: ', str(ra_roc))
print('Resource Allocation Test AP score: ', str(ra_ap))
print('Time taken: {} sec'.format(calc_time)) # this was 9.71s before

Resource Allocation Test ROC score:  0.6041157819008625
Resource Allocation Test AP score:  0.4430262260870411
Time taken: 85.62 sec
